In [53]:
import requests
import json
import random 

URL = "http://localhost:"
ports = range(10000,10006)
def random_url():
    return (URL + str(random.choice(ports))+"/")

def address(addr):
    return str(addr).rjust(16, '0')


### View initial state

In [54]:
initial_utxo = requests.get(random_url() + "utxos/" + address(0)).content.decode('utf-8')
print(f"INITIAL UTXO\n {initial_utxo}")

INITIAL UTXO
 {
  "utxos": [{
    "txId": "0x00000000001",
    "utxoId": "00000000-0000-0000-0000-000000000000",
    "addr": "0000000000000000",
    "coins": "10000"
  }]
}


### Send money between addresses handled by different shards


In [55]:
# from 0 to 1
requests.post(random_url() + "sendCoins", params={"to": address(1), "from": address(0), "coins": "100"})
# from 1 to 2
requests.post(random_url() + "sendCoins", params={"to": address(2), "from": address(1), "coins": "90"})
# from 2 to 3
requests.post(random_url() + "sendCoins", params={"to": address(3), "from": address(2), "coins": "80"})
# from 3 to 4
requests.post(random_url() + "sendCoins", params={"to": address(4), "from": address(3), "coins": "70"})
# from 4 to 1
requests.post(random_url() + "sendCoins", params={"to": address(1), "from": address(4), "coins": "60"})

<Response [200]>

### Get UTxOs after send money series

In [51]:
for addr in range(1, 5):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")


1 UTxOs

{
  "utxos": [{
    "txId": "09102fce-ada2-4cce-bf73-acce5f41c092",
    "utxoId": "7701b637-a73a-3091-b767-9a3e771f2111",
    "addr": "0000000000000001",
    "coins": "10"
  }]
}

2 UTxOs

{
  "utxos": [{
    "txId": "7fef97c4-ca6a-46aa-adf9-25bc056489fb",
    "utxoId": "1dadeed5-90f3-318a-ae54-def7188efe41",
    "addr": "0000000000000002",
    "coins": "10"
  }]
}

3 UTxOs

{
}

4 UTxOs

{
}


### Get ledger history for address 0

In [ ]:
ledger = requests.get(random_url() + "ledger/" + address(0)).content.decode('utf-8')
print(f"Ledger for 0 \n{ledger}")

### Get entire ledger history

In [ ]:
ledger = requests.get(random_url() + "ledger/").content.decode('utf-8')
print(f"{ledger}")

### Make a transaction

Transfer half of the coins for address 1 to address 0

In [ ]:
utxos_1 = json.loads(requests.get(random_url() + "utxos/" + address(0)).content.decode('utf-8'))
coins = sum([int(utxo['coins']) for utxo in utxos_1['utxos']]) 
half_coins = coins //2
output1 = {"addr": address(1), "coins": str(half_coins)}
output2 = {"addr": address(0), "coins": str(coins-half_coins)}
new_tx = {}
new_tx['inputs'] = utxos_1['utxos']
new_tx['outputs'] = output1,output2
new_tx=json.dumps(new_tx).encode('utf-8')
res = requests.post(URL + "submitTx", data=new_tx, headers={'Content-Type': 'application/json'})
print(res.content.decode('utf-8'))

### Print updated UTxOs for 0 and 1 after the transaction

In [ ]:
for addr in range(0, 2):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")